# Order Delivery Time Prediction

## Objectives
The objective of this assignment is to build a regression model that predicts the delivery time for orders placed through Porter. The model will use various features such as the items ordered, the restaurant location, the order protocol, and the availability of delivery partners.

The key goals are:
- Predict the delivery time for an order based on multiple input features
- Improve delivery time predictions to optimiae operational efficiency
- Understand the key factors influencing delivery time to enhance the model's accuracy

## Data Pipeline
The data pipeline for this assignment will involve the following steps:
1. **Data Loading**
2. **Data Preprocessing and Feature Engineering**
3. **Exploratory Data Analysis**
4. **Model Building**
5. **Model Inference**

## Data Understanding
The dataset contains information on orders placed through Porter, with the following columns:

| Field                     | Description                                                                                 |
|---------------------------|---------------------------------------------------------------------------------------------|
| market_id                 | Integer ID representing the market where the restaurant is located.                         |
| created_at                | Timestamp when the order was placed.                                                        |
| actual_delivery_time      | Timestamp when the order was delivered.                                                     |
| store_primary_category    | Category of the restaurant (e.g., fast food, dine-in).                                      |
| order_protocol            | Integer representing how the order was placed (e.g., via Porter, call to restaurant, etc.). |
| total_items               | Total number of items in the order.                                                         |
| subtotal                  | Final price of the order.                                                                   |
| num_distinct_items        | Number of distinct items in the order.                                                      |
| min_item_price            | Price of the cheapest item in the order.                                                    |
| max_item_price            | Price of the most expensive item in the order.                                              |
| total_onshift_dashers     | Number of delivery partners on duty when the order was placed.                              |
| total_busy_dashers        | Number of delivery partners already occupied with other orders.                             |
| total_outstanding_orders  | Number of orders pending fulfillment at the time of the order.                              |
| distance                  | Total distance from the restaurant to the customer.                                         |


## **Importing Necessary Libraries**

In [ ]:
# Import essential libraries for data manipulation and analysis
import pandas as pd




## **1. Loading the data**
Load 'porter_data_1.csv' as a DataFrame

In [ ]:
# Importing the file porter_data_1.csv
df = pd.read_csv('porter_data_1.csv')

# Display first few rows of the dataset
df.head()

## **2. Data Preprocessing and Feature Engineering** <font color = red>[15 marks]</font> <br>

#### **2.1 Fixing the Datatypes**  <font color = red>[5 marks]</font> <br>
The current timestamps are in object format and need conversion to datetime format for easier handling and intended functionality

##### **2.1.1** <font color = red>[2 marks]</font> <br>
Convert date and time fields to appropriate data type

In [ ]:
# Convert 'created_at' and 'actual_delivery_time' columns to datetime format
df['created_at'] = pd.to_datetime(df['created_at'])
df['actual_delivery_time'] = pd.to_datetime(df['actual_delivery_time'])



##### **2.1.2**  <font color = red>[3 marks]</font> <br>
Convert categorical fields to appropriate data type

In [ ]:
# Convert categorical features to category type
categorical_columns = ['store_primary_category', 'order_protocol', 'market_id']
for col in categorical_columns:
    df[col] = df[col].astype('category')



#### **2.2 Feature Engineering** <font color = red>[5 marks]</font> <br>
Calculate the time taken to execute the delivery as well as extract the hour and day at which the order was placed

##### **2.2.1** <font color = red>[2 marks]</font> <br>
Calculate the time taken using the features `actual_delivery_time` and `created_at`

In [ ]:
# Calculate time taken in minutes
df['time_taken'] = (df['actual_delivery_time'] - df['created_at']).dt.total_seconds() / 60

##### **2.2.2** <font color = red>[3 marks]</font> <br>
Extract the hour at which the order was placed and which day of the week it was. Drop the unnecessary columns.

In [ ]:
# Extract the hour and day of week from the 'created_at' timestamp
df['hour'] = df['created_at'].dt.hour
df['day_of_week'] = df['created_at'].dt.dayofweek

# Create a categorical feature 'isWeekend'
df['isWeekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)


In [ ]:
# Drop unnecessary columns
df.drop(columns=['created_at', 'actual_delivery_time'], inplace=True)

#### **2.3 Creating training and validation sets** <font color = red>[5 marks]</font> <br>

##### **2.3.1** <font color = red>[2 marks]</font> <br>
 Define target and input features

In [ ]:
# Define target variable (y) and features (X)
y = df['time_taken']

# Define features (X) - excluding the target variable
X = df.drop(['time_taken'], axis=1)

# Convert categorical variables to dummy variables
X = pd.get_dummies(X, columns=['store_primary_category', 'order_protocol', 'market_id'])



##### **2.3.2** <font color = red>[3 marks]</font> <br>
 Split the data into training and test sets

In [ ]:
# Split data into training and testing sets
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



## **3. Exploratory Data Analysis on Training Data** <font color = red>[20 marks]</font> <br>
1. Analyzing the correlation between variables to identify patterns and relationships
2. Identifying and addressing outliers to ensure the integrity of the analysis
3. Exploring the relationships between variables and examining the distribution of the data for better insights

#### **3.1 Feature Distributions** <font color = red> [7 marks]</font> <br>


In [ ]:
# Define numerical and categorical columns for easy EDA and data manipulation
import seaborn as sns
import matplotlib.pyplot as plt

# Define numerical columns
numerical_columns = ['total_items', 'subtotal', 'num_distinct_items', 
                    'min_item_price', 'max_item_price', 'total_onshift_dashers',
                    'total_busy_dashers', 'total_outstanding_orders', 
                    'distance', 'hour', 'time_taken']

# Define categorical columns 
categorical_columns = ['store_primary_category', 'order_protocol', 
                     'market_id', 'day_of_week', 'isWeekend']



##### **3.1.1** <font color = red>[3 marks]</font> <br>
Plot distributions for numerical columns in the training set to understand their spread and any skewness

In [ ]:
# Plot distributions for all numerical columns
plt.figure(figsize=(20, 15))
plt.suptitle('Distribution of Numerical Features', fontsize=16)

# Plot histograms for each numerical column
for i, col in enumerate(numerical_columns, 1):
    plt.subplot(4, 3, i)
    sns.histplot(data=X_train[col] if col != 'time_taken' else y_train, kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


##### **3.1.2** <font color = red>[2 marks]</font> <br>
Check the distribution of categorical features

In [ ]:
# Distribution of categorical columns
plt.figure(figsize=(15, 10))
plt.suptitle('Distribution of Categorical Features', fontsize=16)

# Plot bar charts for each categorical column
for i, col in enumerate(categorical_columns, 1):
    plt.subplot(2, 3, i)
    value_counts = X_train[col].value_counts() if col in X_train.columns else X_train.filter(like=col).sum()
    sns.barplot(x=value_counts.index, y=value_counts.values)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


##### **3.1.3** <font color = red>[2 mark]</font> <br>
Visualise the distribution of the target variable to understand its spread and any skewness

In [ ]:
# Distribution of time_taken
plt.figure(figsize=(12, 6))
plt.suptitle('Distribution of Delivery Time', fontsize=16)

# Plot histogram with KDE
sns.histplot(data=y_train, kde=True, bins=50)
plt.xlabel('Time Taken (minutes)')
plt.ylabel('Count')

# Add vertical lines for key statistics
plt.axvline(y_train.mean(), color='red', linestyle='--', label=f'Mean: {y_train.mean():.2f}')
plt.axvline(y_train.median(), color='green', linestyle='--', label=f'Median: {y_train.median():.2f}')

plt.legend()
plt.tight_layout()
plt.show()


#### **3.2 Relationships Between Features** <font color = red>[3 marks]</font> <br>

##### **3.2.1** <font color = red>[3 marks]</font> <br>
Scatter plots for important numerical and categorical features to observe how they relate to `time_taken`

In [ ]:
# Scatter plot to visualise the relationship between time_taken and other features
plt.figure(figsize=(20, 12))
plt.suptitle('Relationships between Delivery Time and Key Features', fontsize=16)

# Select important numerical features to plot
important_features = ['distance', 'total_items', 'subtotal', 'total_outstanding_orders']

# Create scatter plots
for i, feature in enumerate(important_features, 1):
    plt.subplot(2, 2, i)
    
    # Create scatter plot with regression line
    sns.regplot(data=X_train, 
                x=feature,
                y=y_train,
                scatter_kws={'alpha':0.5},
                line_kws={'color': 'red'})
    
    plt.title(f'Delivery Time vs {feature}')
    plt.xlabel(feature)
    plt.ylabel('Time Taken (minutes)')

plt.tight_layout()
plt.show()

# Print correlation values
print("\nCorrelations with delivery time:")
for feature in important_features:
    correlation = X_train[feature].corr(y_train)
    print(f"{feature}: {correlation:.3f}")


In [ ]:
# Show the distribution of time_taken for different hours
plt.figure(figsize=(15, 6))
plt.suptitle('Delivery Time Distribution by Hour of Day', fontsize=16)

# Create box plot
sns.boxplot(data=X_train, x='hour', y=y_train)

# Customize the plot
plt.xlabel('Hour of Day')
plt.ylabel('Delivery Time (minutes)')
plt.xticks(range(24))
plt.grid(True, axis='y', linestyle='--', alpha=0.7)

# Add mean delivery time line
mean_time = y_train.mean()
plt.axhline(y=mean_time, color='red', linestyle='--', 
            label=f'Overall Mean: {mean_time:.1f} min')

plt.legend()
plt.tight_layout()
plt.show()

# Print summary statistics by hour
hourly_stats = pd.DataFrame({
    'Mean Time': y_train.groupby(X_train['hour']).mean(),
    'Median Time': y_train.groupby(X_train['hour']).median(),
    'Count': y_train.groupby(X_train['hour']).count()
}).round(2)


#### **3.3 Correlation Analysis** <font color = red>[5 marks]</font> <br>
Check correlations between numerical features to identify which variables are strongly related to `time_taken`

##### **3.3.1** <font color = red>[3 marks]</font> <br>
Plot a heatmap to display correlations

In [ ]:
# Plot the heatmap of the correlation matrix
correlation_matrix = pd.DataFrame(X_train[numerical_columns[:-1]]) # Exclude time_taken
correlation_matrix['time_taken'] = y_train
corr = correlation_matrix.corr()

# Create figure
plt.figure(figsize=(12, 10))
plt.suptitle('Correlation Matrix Heatmap', fontsize=16)

# Create heatmap with annotations
sns.heatmap(corr, 
            annot=True,          # Show correlation values
            cmap='coolwarm',     # Color scheme
            center=0,            # Center the colormap at 0
            square=True,         # Make cells square
            fmt='.2f',           # Format annotations to 2 decimal places
            linewidths=0.5,      # Add gridlines
            cbar_kws={'label': 'Correlation Coefficient'})

plt.tight_layout()
plt.show()

# Print strongest correlations with time_taken
print("\nFeatures most correlated with delivery time:")
correlations = corr['time_taken'].sort_values(ascending=False)
print(correlations.drop('time_taken'))


##### **3.3.2** <font color = red>[2 marks]</font> <br>
Drop the columns with weak correlations with the target variable

In [ ]:
# Drop 3-5 weakly correlated columns from training dataset
correlations = abs(correlations)

# Find features with correlations below threshold (e.g., 0.1)
weak_correlations = correlations[correlations < 0.1].index.tolist()

print("Features with weak correlations:")
for feature in weak_correlations:
    print(f"{feature}: {correlations[feature]:.3f}")

# Drop weakly correlated columns from training and test sets
X_train = X_train.drop(columns=weak_correlations)
X_test = X_test.drop(columns=weak_correlations)


#### **3.4 Handling the Outliers** <font color = red>[5 marks]</font> <br>



##### **3.4.1** <font color = red>[2 marks]</font> <br>
Visualise potential outliers for the target variable and other numerical features using boxplots

In [ ]:
# Boxplot for time_taken
plt.figure(figsize=(15, 10))
plt.suptitle('Distribution and Outliers in Numerical Features', fontsize=16)

# Create boxplots for numerical features including time_taken
all_features = X_train[numerical_columns[:-1]].copy()  # Get numerical features
all_features['time_taken'] = y_train  # Add target variable

# Create boxplot
sns.boxplot(data=all_features)

# Customize plot
plt.xticks(rotation=45)
plt.xlabel('Features')
plt.ylabel('Values')

# Add grid for better readability
plt.grid(True, axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

# Print summary statistics
print("\nSummary statistics for time_taken:")
print(y_train.describe())

# Calculate outlier boundaries for time_taken
Q1 = y_train.quantile(0.25)
Q3 = y_train.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"\nOutlier boundaries for time_taken:")
print(f"Lower bound: {lower_bound:.2f}")
print(f"Upper bound: {upper_bound:.2f}")
print(f"Number of outliers: {len(y_train[(y_train < lower_bound) | (y_train > upper_bound)])}")


##### **3.4.2** <font color = red>[3 marks]</font> <br>
Handle outliers present in all columns

In [ ]:
# Handle outliers
def remove_outliers(df, columns, n_std=1.5):
    """Remove outliers from specified columns using IQR method"""
    df_clean = df.copy()
    for column in columns:
        # Calculate Q1, Q3 and IQR
        Q1 = df_clean[column].quantile(0.25)
        Q3 = df_clean[column].quantile(0.75)
        IQR = Q3 - Q1
        
        # Define outlier bounds
        lower_bound = Q1 - n_std * IQR
        upper_bound = Q3 + n_std * IQR
        
        # Remove outliers
        df_clean = df_clean[
            (df_clean[column] >= lower_bound) & 
            (df_clean[column] <= upper_bound)
        ]
    return df_clean

# Select numerical columns to handle outliers
columns_to_clean = ['time_taken', 'distance', 'total_items', 
                    'subtotal', 'total_outstanding_orders']

# Print original shapes
print("Original shapes:")
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")

# Create DataFrame combining features and target for cleaning
train_data = X_train.copy()
train_data['time_taken'] = y_train

# Remove outliers
train_data_clean = remove_outliers(train_data, columns_to_clean)

# Split back into features and target
X_train_clean = train_data_clean.drop('time_taken', axis=1)
y_train_clean = train_data_clean['time_taken']

# Print new shapes and reduction percentage
print("\nAfter outlier removal:")
print(f"X_train: {X_train_clean.shape}")
print(f"y_train: {y_train_clean.shape}")
print(f"Removed {(len(y_train) - len(y_train_clean))/len(y_train)*100:.1f}% of records")

# Update training data
X_train = X_train_clean
y_train = y_train_clean


## **4. Exploratory Data Analysis on Validation Data** <font color = red>[optional]</font> <br>
Optionally, perform EDA on test data to see if the distribution match with the training data

In [ ]:
# Define numerical and categorical columns for easy EDA and data manipulation
import seaborn as sns
import matplotlib.pyplot as plt

# Define numerical columns (excluding time_taken)
numerical_columns_test = ['total_items', 'subtotal', 'num_distinct_items', 
                         'min_item_price', 'max_item_price', 'total_onshift_dashers',
                         'total_busy_dashers', 'total_outstanding_orders', 
                         'distance', 'hour']

# Define categorical columns (original categories before dummy encoding)
categorical_columns_test = ['store_primary_category', 'order_protocol', 
                          'market_id', 'day_of_week', 'isWeekend']

# Set style for better visualizations
plt.style.use('seaborn')



#### **4.1 Feature Distributions**


##### **4.1.1**
Plot distributions for numerical columns in the validation set to understand their spread and any skewness

In [ ]:
# Plot distributions for all numerical columns

# Plot distributions for numerical columns in test set
plt.figure(figsize=(20, 15))
plt.suptitle('Distribution of Numerical Features (Test Set)', fontsize=16)

# Plot histograms for each numerical column
for i, col in enumerate(numerical_columns_test, 1):
    plt.subplot(4, 3, i)
    sns.histplot(data=X_test[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.xticks(rotation=45)

plt.tight_layout()
plt.show()



##### **4.1.2**
Check the distribution of categorical features

In [ ]:
# Distribution of categorical columns

# Distribution of categorical columns in test set
plt.figure(figsize=(15, 10))
plt.suptitle('Distribution of Categorical Features (Test Set)', fontsize=16)

# Plot bar charts for each categorical column
for i, col in enumerate(categorical_columns_test, 1):
    plt.subplot(2, 3, i)
    value_counts = X_test[col].value_counts() if col in X_test.columns else X_test.filter(like=col).sum()
    sns.barplot(x=value_counts.index, y=value_counts.values)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.xticks(rotation=45)

plt.tight_layout()
plt.show()



##### **4.1.3**
Visualise the distribution of the target variable to understand its spread and any skewness

In [ ]:
# Distribution of time_taken

# Distribution of time_taken in test set
plt.figure(figsize=(12, 6))
plt.suptitle('Distribution of Delivery Time (Test Set)', fontsize=16)

# Plot histogram with KDE
sns.histplot(data=y_test, kde=True, bins=50)
plt.xlabel('Time Taken (minutes)')
plt.ylabel('Count')

# Add vertical lines for key statistics
plt.axvline(y_test.mean(), color='red', linestyle='--', label=f'Mean: {y_test.mean():.2f}')
plt.axvline(y_test.median(), color='green', linestyle='--', label=f'Median: {y_test.median():.2f}')

plt.legend()
plt.tight_layout()
plt.show()

# Compare basic statistics between train and test sets
print('\nComparison of time_taken statistics:')
stats_comparison = pd.DataFrame({
    'Training': y_train.describe(),
    'Test': y_test.describe()
})
print(stats_comparison)



#### **4.2 Relationships Between Features**
Scatter plots for numerical features to observe how they relate to each other, especially to `time_taken`

In [ ]:
# Scatter plot to visualise the relationship between time_taken and other features

# Scatter plots for key features in test set
plt.figure(figsize=(20, 12))
plt.suptitle('Relationships between Delivery Time and Key Features (Test Set)', fontsize=16)

# Select important numerical features
important_features = ['distance', 'total_items', 'subtotal', 'total_outstanding_orders']

# Create scatter plots
for i, feature in enumerate(important_features, 1):
    plt.subplot(2, 2, i)
    sns.regplot(data=X_test, 
                x=feature,
                y=y_test,
                scatter_kws={'alpha':0.5},
                line_kws={'color': 'red'})
    plt.title(f'Delivery Time vs {feature}')
    plt.xlabel(feature)
    plt.ylabel('Time Taken (minutes)')

plt.tight_layout()
plt.show()

# Print correlation values
print("\nCorrelations with delivery time (Test Set):")
for feature in important_features:
    correlation = X_test[feature].corr(y_test)
    print(f"{feature}: {correlation:.3f}")



#### **4.3** Drop the columns with weak correlations with the target variable

In [ ]:
# Drop the weakly correlated columns from training dataset

# Calculate correlations in test set
test_correlation_matrix = pd.DataFrame(X_test[numerical_columns_test])
test_correlation_matrix['time_taken'] = y_test
test_corr = abs(test_correlation_matrix.corr()['time_taken'])

# Find features with weak correlations
weak_correlations_test = test_corr[test_corr < 0.1].index.tolist()

print("Features with weak correlations in test set:")
for feature in weak_correlations_test:
    print(f"{feature}: {test_corr[feature]:.3f}")

# Compare weak correlations between train and test
print("\nComparison of weak correlations:")
print(f"Train set: {weak_correlations}")
print(f"Test set: {weak_correlations_test}")



## **5. Model Building** <font color = red>[15 marks]</font> <br>

#### **Import Necessary Libraries**

In [ ]:
# Import libraries
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from scipy import stats



#### **5.1 Feature Scaling** <font color = red>[3 marks]</font> <br>

In [ ]:
# Apply scaling to the numerical columns

# Initialize the scaler
scaler = StandardScaler()

# Get numerical columns
numerical_features = X_train.select_dtypes(include=['float64', 'int64']).columns

# Scale numerical features
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[numerical_features] = scaler.fit_transform(X_train[numerical_features])
X_test_scaled[numerical_features] = scaler.transform(X_test[numerical_features])

print('Scaled features:', numerical_features.tolist())



Note that linear regression is agnostic to feature scaling. However, with feature scaling, we get the coefficients to be somewhat on the same scale so that it becomes easier to compare them.

#### **5.2 Build a linear regression model** <font color = red>[5 marks]</font> <br>

You can choose from the libraries *statsmodels* and *scikit-learn* to build the model.

In [ ]:
# Create/Initialise the model

# Initialize the linear regression model
model = LinearRegression()

# Initialize RFE with 8 features
selector = RFE(estimator=model, n_features_to_select=8, step=1)

# Fit RFE
selector = selector.fit(X_train_scaled, y_train)

# Get selected features
selected_features = X_train_scaled.columns[selector.support_].tolist()
print('Selected features:', selected_features)


In [ ]:
# Train the model using the training data

# Train model with selected features
final_model = LinearRegression()
final_model.fit(X_train_scaled[selected_features], y_train)

# Print model summary
coef_df = pd.DataFrame({
    'Feature': selected_features,
    'Coefficient': final_model.coef_
})
print('Model Coefficients:')
print(coef_df.sort_values(by='Coefficient', ascending=False))
print(f'\nIntercept: {final_model.intercept_:.2f}')


In [ ]:
# Make predictions on train and test sets
y_train_pred = final_model.predict(X_train_scaled[selected_features])
y_test_pred = final_model.predict(X_test_scaled[selected_features])

# Calculate residuals
train_residuals = y_train - y_train_pred
test_residuals = y_test - y_test_pred


In [ ]:
# Find results for evaluation metrics

# Calculate performance metrics
train_mse = mean_squared_error(y_train, y_train_pred)
train_rmse = np.sqrt(train_mse)
train_r2 = r2_score(y_train, y_train_pred)

test_mse = mean_squared_error(y_test, y_test_pred)
test_rmse = np.sqrt(test_mse)
test_r2 = r2_score(y_test, y_test_pred)

print('Training Metrics:')
print(f'RMSE: {train_rmse:.2f}')
print(f'R²: {train_r2:.3f}')

print('\nTest Metrics:')
print(f'RMSE: {test_rmse:.2f}')
print(f'R²: {test_r2:.3f}')



Note that we have 12 (depending on how you select features) training features. However, not all of them would be useful. Let's say we want to take the most relevant 8 features.

We will use Recursive Feature Elimination (RFE) here.

For this, you can look at the coefficients / p-values of features from the model summary and perform feature elimination, or you can use the RFE module provided with *scikit-learn*.

#### **5.3 Build the model and fit RFE to select the most important features** <font color = red>[7 marks]</font> <br>

For RFE, we will start with all features and use
the RFE method to recursively reduce the number of features one-by-one.

After analysing the results of these iterations, we select the one that has a good balance between performance and number of features.

In [ ]:
# Loop through the number of features and test the model



In [ ]:
# Build the final model with selected number of features



## **6. Results and Inference** <font color = red>[5 marks]</font> <br>

#### **6.1 Perform Residual Analysis** <font color = red>[3 marks]</font> <br>

In [ ]:
# Perform residual analysis using plots like residuals vs predicted values, Q-Q plot and residual histogram

# Create figure for residual plots
plt.figure(figsize=(15, 10))

# 1. Residuals vs Predicted Values
plt.subplot(2, 2, 1)
plt.scatter(y_train_pred, train_residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted Values')

# 2. Residual Histogram
plt.subplot(2, 2, 2)
plt.hist(train_residuals, bins=30, edgecolor='black')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Residual Histogram')

# 3. Q-Q Plot
plt.subplot(2, 2, 3)
stats.probplot(train_residuals, dist='norm', plot=plt)
plt.title('Q-Q Plot')

# 4. Residuals vs Order
plt.subplot(2, 2, 4)
plt.plot(train_residuals, marker='o', linestyle='none', alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Order')
plt.ylabel('Residuals')
plt.title('Residuals vs Order')

plt.tight_layout()
plt.show()

# Print residual statistics
print('Residual Statistics:')
print(f'Mean: {np.mean(train_residuals):.2f}')
print(f'Std Dev: {np.std(train_residuals):.2f}')



[Your inferences here:]



#### **6.2 Perform Coefficient Analysis** <font color = red>[2 marks]</font> <br>

Perform coefficient analysis to find how changes in features affect the target.
Also, the features were scaled, so interpret the scaled and unscaled coefficients to understand the impact of feature changes on delivery time.


In [ ]:
# Compare the scaled vs unscaled features used in the final model

# Compare scaled vs unscaled coefficients
scaled_coef = pd.DataFrame({
    'Feature': selected_features,
    'Scaled_Coefficient': final_model.coef_,
    'Abs_Impact': abs(final_model.coef_)
}).sort_values('Abs_Impact', ascending=False)

# Calculate unscaled coefficients
unscaled_coef = final_model.coef_ / scaler.scale_[numerical_features.isin(selected_features)]
unscaled_coef = pd.DataFrame({
    'Feature': [f for f in selected_features if f in numerical_features],
    'Unscaled_Coefficient': unscaled_coef
})

print('Feature Importance (Scaled Coefficients):')
print(scaled_coef)

print('\nUnscaled Coefficients (Original Units):')
print(unscaled_coef)



Additionally, we can analyse the effect of a unit change in a feature. In other words, because we have scaled the features, a unit change in the features will not translate directly to the model. Use scaled and unscaled coefficients to find how will a unit change in a feature affect the target.

In [ ]:
# Analyze the effect of a unit change in a feature, say 'total_items'

# Analyze effect of unit change in important features
for feature in ['total_items', 'distance', 'total_outstanding_orders']:
    if feature in numerical_features:
        # Get the scaled coefficient
        coef = final_model.coef_[selected_features.index(feature)]
        # Get the scaling parameters
        scale = scaler.scale_[numerical_features.get_loc(feature)]
        mean = scaler.mean_[numerical_features.get_loc(feature)]
        
        # Calculate impact of one unit change
        unit_impact = coef / scale
        
        print(f'\nFeature: {feature}')
        print(f'One unit increase leads to {unit_impact:.2f} minute change in delivery time')
        print(f'Mean value: {mean:.2f}')



Note:
The coefficients on the original scale might differ greatly in magnitude from the scaled coefficients, but they both describe the same relationships between variables.

Interpretation is key: Focus on the direction and magnitude of the coefficients on the original scale to understand the impact of each variable on the response variable in the original units.

Include conclusions in your report document.

## Subjective Questions <font color = red>[20 marks]</font>

Answer the following questions only in the notebook. Include the visualisations/methodologies/insights/outcomes from all the above steps in your report.

#### Subjective Questions based on Assignment

##### **Question 1.** <font color = red>[2 marks]</font> <br>

Are there any categorical variables in the data? From your analysis of the categorical variables from the dataset, what could you infer about their effect on the dependent variable?

**Answer:**
> Yes, there are several categorical variables in the data:
> 1. store_primary_category (restaurant type)
> 2. order_protocol (order placement method)
> 3. market_id (location identifier)
> 4. day_of_week and isWeekend (derived temporal features)
>
> From one-hot encoding and subsequent analysis, we can see these variables contribute to delivery time variations. Different restaurant categories and protocols show distinct delivery time patterns, while market locations impact transit times.



---



##### **Question 2.** <font color = red>[1 marks]</font> <br>
What does `test_size = 0.2` refer to during splitting the data into training and test sets?

**Answer:**
> test_size = 0.2 means that 20% of the dataset is reserved for testing while 80% is used for training. In other words, if we have 1000 samples, 800 will be used for training the model and 200 for testing its performance.



---



##### **Question 3.** <font color = red>[1 marks]</font> <br>
Looking at the heatmap, which one has the highest correlation with the target variable?  

**Answer:**
> Looking at the correlation heatmap, 'distance' has the highest correlation with the target variable 'time_taken'. This makes intuitive sense as longer distances typically require more delivery time.



---



##### **Question 4.** <font color = red>[2 marks]</font> <br>
What was your approach to detect the outliers? How did you address them?

**Answer:**

>
We used the Interquartile Range (IQR) method to detect outliers:
1. Calculated Q1 (25th percentile) and Q3 (75th percentile)
2. Computed IQR = Q3 - Q1
3. Defined outliers as values outside [Q1 - 1.5*IQR, Q3 + 1.5*IQR]

We addressed outliers by removing records that fell outside these boundaries for key numerical features (time_taken, distance, total_items, subtotal, total_outstanding_orders).



---



##### **Question 5.** <font color = red>[2 marks]</font> <br>
Based on the final model, which are the top 3 features significantly affecting the delivery time?

**Answer:**
> The top 3 features significantly affecting delivery time are:
> 1. Distance (strongest positive correlation)
> 2. Total outstanding orders (indicates system load)
> 3. Total items (order size)
> These features show the strongest correlations and highest coefficient magnitudes in the final model.



---



#### General Subjective Questions

##### **Question 6.** <font color = red>[3 marks]</font> <br>
Explain the linear regression algorithm in detail

**Answer:**
> Linear regression is a supervised learning algorithm that models the relationship between a dependent variable (y) and one or more independent variables (x) by fitting a linear equation:
> y = β₀ + β₁x₁ + β₂x₂ + ... + βₙxₙ + ε
>
> The algorithm:
> 1. Takes input features and target variable
> 2. Assumes a linear relationship
> 3. Uses Ordinary Least Squares (OLS) to minimize the sum of squared residuals
> 4. Finds optimal coefficients (β) that best fit the data
> 5. Creates a model that can predict target values for new inputs




---



##### **Question 7.** <font color = red>[2 marks]</font> <br>
Explain the difference between simple linear regression and multiple linear regression

**Answer:**
> Simple Linear Regression:
> - Uses only one independent variable (x)
> - Models relationship as y = β₀ + β₁x + ε
> - Visualized as a straight line in 2D
>
> Multiple Linear Regression:
> - Uses two or more independent variables
> - Models as y = β₀ + β₁x₁ + β₂x₂ + ... + βₙxₙ + ε
> - Represents a hyperplane in multi-dimensional space



---



##### **Question 8.** <font color = red>[2 marks]</font> <br>
What is the role of the cost function in linear regression, and how is it minimized?

**Answer:**
> The cost function in linear regression, typically Mean Squared Error (MSE), measures how well the model fits the data:
> MSE = (1/n)Σ(yᵢ - ŷᵢ)²
>
> It's minimized through:
> 1. Gradient Descent: Iteratively adjusts coefficients
> 2. Ordinary Least Squares: Analytical solution for minimum
> 3. The goal is to find coefficients that produce the smallest possible MSE




---



##### **Question 9.** <font color = red>[2 marks]</font> <br>
Explain the difference between overfitting and underfitting.



**Answer:**

>
> Overfitting:
> - Model learns noise in training data
> - High training accuracy but poor generalization
> - Too complex, captures random fluctuations
>
> Underfitting:
> - Model fails to capture underlying patterns
> - Poor performance on both training and test data
> - Too simple to represent the relationship



---



##### **Question 10.** <font color = red>[3 marks]</font> <br>
How do residual plots help in diagnosing a linear regression model?

**Answer:**
> Residual plots help diagnose linear regression models by revealing:
> 1. Linearity: Pattern in residuals suggests non-linear relationships
> 2. Homoscedasticity: Consistent variance across predictions
> 3. Normality: Through Q-Q plots of residuals
> 4. Outliers: Points with large residuals
> 5. Model adequacy: Random scatter indicates good fit